<a href="https://colab.research.google.com/github/cwscx/nanogpt/blob/main/gpt_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-08-24 18:14:32--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.2’

input.txt.2         100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2025-08-24 18:14:32 (16.5 MB/s) - ‘input.txt.2’ saved [1115394/1115394]



In [65]:
with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [66]:
print(len(text))

1115394


In [67]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [68]:
sorted_chars: int = sorted(list(set(text)))
vocab_size: int = len(sorted_chars)
print(''.join(sorted_chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [69]:
charToIndex = {}
indexToChar = {}
for i, char in enumerate(sorted_chars):
  charToIndex[char] = i;
  indexToChar[i] = char

def encode(text: str) -> list[int]:
  encoded = []
  for char in text:
    encoded.append(charToIndex[char])
  return encoded

def decode(encoded: list[int]) -> str:
  chars = []
  for token in encoded:
    chars.append(indexToChar[token])
  return ''.join(chars)

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [70]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [71]:
n = int(len(text) * 0.9)
training = data[:n]
validation = data[n:]

In [72]:
block_size = 8
# The reason to have + 1 is that the model is going to predict the next token.
# So based on char at index i, predirct next char at index i + 1.
# So block_size + 1 dataset contains block_size number of char pairs [i, i + 1]
# to predict.
train_data = training[:block_size + 1]

In [73]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]
for i in range(block_size):
  context: torch.tensor = train_data[:i + 1]  # i + 1 is not inclusive
  target: int = train_data[i + 1]
  print(f"when input is {context} target is {target}")

when input is tensor([18]) target is 47
when input is tensor([18, 47]) target is 56
when input is tensor([18, 47, 56]) target is 57
when input is tensor([18, 47, 56, 57]) target is 58
when input is tensor([18, 47, 56, 57, 58]) target is 1
when input is tensor([18, 47, 56, 57, 58,  1]) target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) target is 58


In [74]:
torch.manual_seed(1337)   # Make outcome deterministic
batch_size = 4 # how many sequence we are going to process in parallel
block_size = 8 # maximum context length to predict the next char

def get_batch(split: str) -> tuple[torch.Tensor, torch.Tensor]:
  if split == 'train':
    data = training
  else:
    data = validation

  ## 4 random generated offsets range from [0, 1115394].
  offsets: list[int] = torch.randint(len(data) - block_size, (batch_size, ))

  x = torch.stack([data[i:i + block_size] for i in offsets])
  y = torch.stack([data[i + 1:i + block_size + 1] for i in offsets])
  return x, y

xb, yb = get_batch("train")  ## inputs & targets
print("inputs", xb, xb.shape) ## 4 by 8 tensor.
print("targets", yb, yb.shape)

for b in range(batch_size):
  for t in range(block_size):
    context = xb[b][:t+1]
    target = yb[b][t]
    print(f"when input is {context}, the target is {target}")

inputs tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]]) torch.Size([4, 8])
targets tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]]) torch.Size([4, 8])
when input is tensor([24]), the target is 43
when input is tensor([24, 43]), the target is 58
when input is tensor([24, 43, 58]), the target is 5
when input is tensor([24, 43, 58,  5]), the target is 57
when input is tensor([24, 43, 58,  5, 57]), the target is 1
when input is tensor([24, 43, 58,  5, 57,  1]), the target is 46
when input is tensor([24, 43, 58,  5, 57,  1, 46]), the target is 43
when input is tensor([24, 43, 58,  5, 57,  1, 46, 43]), the target is 39
when input is tensor([44]), the target is 53
when input is tensor([44, 53]), the target is 56
when input is tensor([44, 53, 56]), the targe

In [75]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

  def __init__(self, vocab_size: int) -> None:
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # A simple lookup table that stores embeddings of a fixed dictionary and size.

  def forward(self, idx: list[int], targets: torch.tensor=None) -> tuple[torch.tensor, torch.tensor]:
    logits = self.token_embedding_table(idx) # generates a tensor (batch x time x channel)
    # batch = 4, time = 8 (the block size), channel = vocab_size

    if targets is None:
      loss = None
    else:
      B,T,C = logits.shape
      logits = logits.view(B*T,C) # Flatten B,C to make the shape callable to cross_entropy
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)
      # print(loss)

    return logits, loss

  def generate(self, idx: list[int], max_new_tokens: int):
    for _ in range(max_new_tokens):
      logits, loss = self(idx) # pytorch nn.Module define __call__ which will call forward method.
      logits = logits[:, -1, :] # : take all，-1 take last. Make logits tensor shape [B, C] => [4, 65]
      probs = F.softmax(logits, dim=1) # run softmax on second dimension in logits. The sum of probs will be 1.
      idx_next = torch.multinomial(probs, num_samples=1)  # pick one randomly according to prob. shape is [B, 1]
      idx=torch.cat((idx, idx_next), dim=1) # generate next token.
    return idx


m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
# print(logits.shape)
# print(loss)

idx = torch.zeros((1,1), dtype=torch.long)
print(idx)
print('wtf', m.generate(idx, max_new_tokens=100))
print('wtf', m.generate(idx, max_new_tokens=100)[0])
print('decode', decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

tensor([[0]])
wtf tensor([[ 0, 31, 23, 21, 41, 24, 32, 11, 13, 41, 17, 24, 25, 53, 32, 40, 60, 38,
         60,  1, 15, 12, 52, 55,  7, 29, 17,  9,  9, 10, 15, 22, 55, 49, 27, 23,
         20,  7, 55, 11, 10, 50, 39,  2, 53, 47, 63, 61, 49, 20, 48, 45, 15, 46,
         64, 40, 29, 12, 59,  2,  9, 40, 24, 21, 45, 61, 43, 60, 51, 63, 18, 22,
         19, 33, 19, 54,  0, 61, 52, 37, 35, 51, 52, 62, 23, 35, 35, 43, 60,  7,
         58, 16, 55, 36, 17, 56, 34, 23, 24, 45, 22]])
wtf tensor([ 0, 54, 22, 10, 14, 54, 51,  4, 63, 47, 50, 58, 26, 15, 48, 43, 27,  9,
        10, 15, 62,  4, 60, 60, 25, 37, 35,  7, 58, 62, 48, 59, 13, 42,  1, 21,
        30, 18, 40, 32, 54, 22,  3, 64, 49, 38, 43, 50, 62, 38, 58, 32, 50, 20,
        26, 64, 42, 36, 36, 33, 47, 29, 29, 37, 10, 55, 18, 21, 26, 32, 27, 14,
        26, 24, 21,  6,  4, 53, 32, 47, 45, 55,  1, 64,  8, 41, 10, 15, 55,  6,
        31, 16, 36, 64, 43, 58, 52,  9, 36, 34, 48])
decode 
SBLBcHAUhk&PHdhcOb
nhJ?FJU?pRiOLQeUN!BxjPLiq-GJdUV'hsnla!

In [76]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3) # lr for learning rate

In [81]:
batch_size = 32

for steps in range(10000):
  xb, yb = get_batch('train')

  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())


2.524625539779663


In [85]:
print('decode', decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=500)[0].tolist()))

decode 
Bovuind y h wit hesanPUEOFr t per

Got'anithe toberld, Tht, m. leve.

Anins, me ot thteatinksowaitt y ave theloly brice hour e t ce Ifoupie winovejed, g wok hes ms wicadove mitin, CIS:
Fa stou m ty, lllin INCI furepe,
Whaborlere le atrvicthent. f wat is ure tey
Be, theadery s, INCHene falllpo houale akengratho thes H3Th


WIIst s nd thousoffose alle las l f pecce.
Whandefun w leim wain Fote ind.
Poligagin ndurrd br d. tes, f l fe gim ve tor akin'ss s;
ver btino!
thif ithap p lorslithouthioupl bo
